In [1]:
import sys
!{sys.executable} -m pip install pandas

     |████████████████████████████████| 11.5 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.8 MB 4.3 MB/s eta 0:00:01    |██████████████▋                 | 7.2 MB 4.4 MB/s eta 0:00:02
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
from collections import Counter
import json
import os

import pandas as pd

In [3]:
df = pd.read_csv('temp/tweets.csv')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
len(df)

410874

In [5]:
df.head()

,id,date,username,tweet
0,1399401864735727617,2021-05-31,foreignpoli_c,Canadian military trainers and aid have suppor...
1,1398705022045671427,2021-05-29,foreignpoli_c,@davidsheen In Canada JDL's Weinstein coordina...
2,1398700432399155200,2021-05-29,foreignpoli_c,Stop all Canadian weapons sales. Remove Colomb...
3,1398680866474409996,2021-05-29,foreignpoli_c,Liberal MP Nathaniel Erskine-Smith openly crit...
4,1398666728557539329,2021-05-29,foreignpoli_c,As projected costs have ballooned to $286 bill...


In [6]:
df = df.sample(500, random_state=42)

In [7]:
df.head()

,id,date,username,tweet
103923,1292567579098308609,2020-08-09,sputnikint,Pentagon chief explains why US is putting more...
170254,1314833041731223552,2020-10-10,cgtnofficial,Miami #Heat edge Los Angeles #Lakers in thrill...
71393,1352055812408033280,2021-01-21,sputnikint,NASA may limit its presence in Russia over shr...
37567,1359901561032425474,2021-02-11,nationalpost,Ottawa announces $55 million in clean tech fun...
269573,1371580152258433025,2021-03-15,rt_com,Portugal's top court rejects bill to legalize ...


In [8]:
def tokenize_and_clean(text):
    tokens = text.split(' ')
    tokens = [x.replace('#', '').lower() 
              for x in tokens]
    tokens = [x for x in tokens
              if x != ''
              and not x.startswith('http')]
    return tokens

In [9]:
for _, x in df.iloc[0:5].iterrows():
    print(x.tweet)
    print(tokenize_and_clean(x.tweet))

Pentagon chief explains why US is putting more troops on Russia’s borders  @DeptofDefense @EsperDoD  https://t.co/rmLeRu5Fpu
['pentagon', 'chief', 'explains', 'why', 'us', 'is', 'putting', 'more', 'troops', 'on', 'russia’s', 'borders', '@deptofdefense', '@esperdod']
Miami #Heat edge Los Angeles #Lakers in thriller to keep #NBA title dream alive  https://t.co/8rtZcSFtax
['miami', 'heat', 'edge', 'los', 'angeles', 'lakers', 'in', 'thriller', 'to', 'keep', 'nba', 'title', 'dream', 'alive']
NASA may limit its presence in Russia over shrinking cooperation on ISS, sources say  @NASA @Space_Station  https://t.co/fC6bWOQyYp
['nasa', 'may', 'limit', 'its', 'presence', 'in', 'russia', 'over', 'shrinking', 'cooperation', 'on', 'iss,', 'sources', 'say', '@nasa', '@space_station']
Ottawa announces $55 million in clean tech funding for 20 companies  https://t.co/0c9H35MCYb
['ottawa', 'announces', '$55', 'million', 'in', 'clean', 'tech', 'funding', 'for', '20', 'companies']
Portugal's top court rejec

In [10]:
for _, x in df.iterrows():
    with open(f'data/tweets/{x.id}.txt', 'w+') as f:
        tokens = tokenize_and_clean(x.tweet)
        f.write(' '.join(tokens))

## Data Loader, Vocab, Etc

In [18]:
def get_docs():
    files = os.listdir('data/tweets')
    for file in files:
        with open(f'data/tweets/{file}') as f:
            tokens = f.read().strip().split(' ')
            if len(tokens) >= 5:
                yield tokens

def get_tokens():
    for doc in get_docs():
        for token in doc:
            yield token

In [19]:
vocab = list(set(get_tokens()))
vocab = dict(zip(vocab, range(len(vocab))))
token_to_count = {t: c for t, c in Counter(get_tokens()).items()}

In [20]:
with open('data/vocab.json', 'w+') as f:
    f.write(json.dumps(vocab))

In [21]:
with open('data/counts.json', 'w+') as f:
    f.write(json.dumps(token_to_count))

In [22]:
contexts = {}
window = 2

for doc in get_docs():
    for i in range(len(doc)):
        tok = doc[i]
        ctx_start = max(0, i - window)
        ctx_end = min(len(doc) - 1, i + window)
        ctx_ixs = [j for j in range(ctx_start, ctx_end)
                   if j != i]
        if tok not in contexts:
            contexts[tok] = set([])
        ctxs = [doc[j] for j in ctx_ixs]
        contexts[tok].update(ctxs)

In [24]:
contexts = {t: list(c) for t, c in contexts.items()}

In [25]:
min([len(v) for v in contexts.values()])

1

In [26]:
with open('data/contexts.json', 'w+') as f:
    f.write(json.dumps(contexts))

In [30]:
contexts['biden']

['administration',
 'for',
 'incoming',
 'announces',
 'joe',
 'pressure',
 'the',
 'to',
 'immediately',
 'certified',
 'president',
 'if']